##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFLite コードを TF2 に移行する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tflite">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/tflite.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/tflite.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

[TensorFlow Lite](https://www.tensorflow.org/lite/guide)（TFLite）は、開発者がデバイス（モバイル、組み込み、IoT デバイス）で ML 推論を実行するのに役立つ一連のツールです。[TFLite コンバータ](https://www.tensorflow.org/lite/convert)は、既存の TF モデルをデバイス上で効率的に実行できる最適化された TFLite モデル形式に変換するツールの 1 つです。

このドキュメントでは、TF から TFLite への変換コードにどのような変更を加える必要があるかを説明し、いくつかの例を示します。

## TF から TFLite への変換コードの変更

- 従来の TF1 モデル形式（Keras ファイル、凍結された GraphDef、チェックポイント、tf.Session など）を使用している場合は、それを TF1/TF2 SavedModel に更新し、TF2 コンバータ API `tf.lite.TFLiteConverter.from_saved_model(...)` を使用して TFLite モデルに変換します（表 1 を参照）。

- コンバータ API フラグを更新します（表 2 を参照）。

- `tflite.constants` などのレガシー API を削除します。（例: `tf.lite.constants.INT8` を `tf.int8` に置き換えます）

// 表 1 // TFLite Python コンバータ API の更新

TF1 API | TF2 API
--- | ---
`tf.lite.TFLiteConverter.from_saved_model('saved_model/',..)` | *サポートされています*
`tf.lite.TFLiteConverter.from_keras_model_file('model.h5',..)` | *削除（SavedModel 形式に更新）*
`tf.lite.TFLiteConverter.from_frozen_graph('model.pb',..)` | *削除（SavedModel 形式に更新）*
`tf.lite.TFLiteConverter.from_session(sess,...)` | *削除（SavedModel 形式に更新）*

&lt;style&gt;   .table {margin-left: 0 !important;} &lt;/style&gt;

// 表 2 // TFLite Python コンバータ API フラグの更新

TF1 API | TF2 API
--- | ---
`allow_custom_ops`<br>`optimizations`<br> `representative_dataset`<br>`target_spec` <br>`inference_input_type`<br>`inference_output_type`<br>`experimental_new_converter`<br> `experimental_new_quantizer` | *サポートされています* <br><br><br><br><br><br><br><br>
`input_tensors`<br>`output_tensors`<br>`input_arrays_with_shape`<br>`output_arrays`<br>`experimental_debug_info_func` | *削除（コンバータ API 引数がサポートされていません）*<br><br><br><br><br>
`change_concat_input_ranges`<br>`default_ranges_stats`<br>`get_input_arrays()`<br>`inference_type`<br>`quantized_input_stats`<br> `reorder_across_fake_quant` | *削除（量子化ワークフローがサポートされていません）*<br><br><br><br><br><br>
`conversion_summary_dir`<br>`dump_graphviz_dir`<br>`dump_graphviz_video` | *削除（モデルを可視化するには、[Netron](https://lutzroeder.github.io/netron/) または [visualize.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/tools/visualize.py) を使用します）*<br><br><br>
`output_format`<br>`drop_control_dependency` | *削除（TF2 でサポートされていない機能）*<br><br>

## 例

ここでは、レガシー TF1 モデルを TF1/TF2 SavedModel に変換し、それらを TF2 TFLite モデルに変換するいくつかの例について説明します。

### セットアップ

必要な TensorFlow インポートから始めます。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

import shutil
def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass

必要なすべての TF1 モデル形式を作成します。

In [ ]:
# Create a TF1 SavedModel
SAVED_MODEL_DIR = "tf_saved_model/"
remove_dir(SAVED_MODEL_DIR)
with tf1.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=(3,), name='input')
    output = input + 2
    # print("result: ", sess.run(output, {input: [0., 2., 4.]}))
    tf1.saved_model.simple_save(
        sess, SAVED_MODEL_DIR,
        inputs={'input': input}, 
        outputs={'output': output})
print("TF1 SavedModel path: ", SAVED_MODEL_DIR)

# Create a TF1 Keras model
KERAS_MODEL_PATH = 'tf_keras_model.h5'
model = tf1.keras.models.Sequential([
    tf1.keras.layers.InputLayer(input_shape=(128, 128, 3,), name='input'),
    tf1.keras.layers.Dense(units=16, input_shape=(128, 128, 3,), activation='relu'),
    tf1.keras.layers.Dense(units=1, name='output')
])
model.save(KERAS_MODEL_PATH, save_format='h5')
print("TF1 Keras Model path: ", KERAS_MODEL_PATH)

# Create a TF1 frozen GraphDef model
GRAPH_DEF_MODEL_PATH = tf.keras.utils.get_file(
    'mobilenet_v1_0.25_128',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.25_128_frozen.tgz',
    untar=True,
) + '/frozen_graph.pb'

print("TF1 frozen GraphDef path: ", GRAPH_DEF_MODEL_PATH)

### 1. TF1 SavedModel を TFLite モデルに変換する


#### 以前: TF1 で変換する

以下は、TF1 スタイルの TFlite 変換の典型的なコードです。


In [ ]:
converter = tf1.lite.TFLiteConverter.from_saved_model(
    saved_model_dir=SAVED_MODEL_DIR,
    input_arrays=['input'],
    input_shapes={'input' : [3]}
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()
# Ignore warning: "Use '@tf.function' or '@defun' to decorate the function."

#### 更新後: TF2 で変換する

TF1 SavedModel を TFLite モデルに直接変換し、より小さい v2 コンバータフラグを設定します。

In [ ]:
# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=SAVED_MODEL_DIR)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

### 2. TF1 Keras モデルファイルを TFLite モデルに変換する

#### 以前: TF1 で変換する

これは、TF1 スタイルの TFlite 変換の典型的なコードです。

In [ ]:
converter = tf1.lite.TFLiteConverter.from_keras_model_file(model_file=KERAS_MODEL_PATH)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### 更新後: TF2 で変換する

TF1 Keras モデルファイルを TF2 SavedModel に変換してから、より小さな v2 コンバータフラグを設定し、それを TFLite モデルに変換します。

In [ ]:
# Convert TF1 Keras model file to TF2 SavedModel.
model = tf.keras.models.load_model(KERAS_MODEL_PATH)
model.save(filepath='saved_model_2/')

# Convert TF2 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_2/')
tflite_model = converter.convert()

### 3. TF1 で凍結された GraphDef を TFLite モデルに変換する


#### 以前: TF1 で変換する

これは、TF1 スタイルの TFlite 変換の典型的なコードです。

In [ ]:
converter = tf1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=GRAPH_DEF_MODEL_PATH,
    input_arrays=['input'],
    input_shapes={'input' : [1, 128, 128, 3]},
    output_arrays=['MobilenetV1/Predictions/Softmax'],
)
converter.optimizations = {tf.lite.Optimize.DEFAULT}
converter.change_concat_input_ranges = True
tflite_model = converter.convert()

#### 更新後: TF2 で変換する

TF1 で凍結された GraphDef を TF1 SavedModel に変換してから、より小さな v2 コンバータフラグを設定して、それを TFLite モデルに変換します。


In [ ]:
## Convert TF1 frozen Graph to TF1 SavedModel.

# Load the graph as a v1.GraphDef
import pathlib
gdef = tf.compat.v1.GraphDef()
gdef.ParseFromString(pathlib.Path(GRAPH_DEF_MODEL_PATH).read_bytes())

# Convert the GraphDef to a tf.Graph
with tf.Graph().as_default() as g:
  tf.graph_util.import_graph_def(gdef, name="")

# Look up the input and output tensors.
input_tensor = g.get_tensor_by_name('input:0') 
output_tensor = g.get_tensor_by_name('MobilenetV1/Predictions/Softmax:0')

# Save the graph as a TF1 Savedmodel
remove_dir('saved_model_3/')
with tf.compat.v1.Session(graph=g) as s:
  tf.compat.v1.saved_model.simple_save(
      session=s,
      export_dir='saved_model_3/',
      inputs={'input':input_tensor},
      outputs={'output':output_tensor})

# Convert TF1 SavedModel to a TFLite model.
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_3/')
converter.optimizations = {tf.lite.Optimize.DEFAULT}
tflite_model = converter.convert()

# 参考資料

- ワークフローと最新機能の詳細については、[TFLite ガイド](https://www.tensorflow.org/lite/guide)を参照してください。
- TF1 コードまたは従来の TF1 モデル形式（Keras `.h5` ファイル、凍結された GraphDef `.pb` など）を使用している場合、コードを更新し、モデルを [TF2 SavedModel 形式](https://www.tensorflow.org/guide/saved_model)に移行してください。
